# Lab-3: Null value handling emploing severel techniques

# Paractice a Problem

In [3]:
#openjdk installation
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Downloading the apche spark with specific version
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
#Zip into unzip the apche spark
!tar -xf spark-3.4.1-bin-hadoop3.tgz
#findspark installation
!pip install -q findspark
#Environment setup for the openjdk and spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [11]:
# Upload the file
from google.colab import files
files.upload()

Saving lab3.csv to lab3 (1).csv


{'lab3 (1).csv': b'Name,Age,Experience,Salary\nMousumi,50,20,30000\n,,,\nNahid,22,5,10000\nSalauddin,24,12,50000\nDurjoy,18,,\nAsha,,,\nMafiul,27,14,60000\n'}

In [21]:
df = spark.read.format('csv').option("header","true").option("inferschema","true").option("mode","failfast").load("lab3 (1).csv")

In [22]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [17]:
df.show()

+---------+----+----------+------+
|     Name| Age|Experience|Salary|
+---------+----+----------+------+
|  Mousumi|  50|        20| 30000|
|     null|null|      null|  null|
|    Nahid|  22|         5| 10000|
|Salauddin|  24|        12| 50000|
|   Durjoy|  18|      null|  null|
|     Asha|null|      null|  null|
|   Mafiul|  27|        14| 60000|
+---------+----+----------+------+



In [18]:
type(df)

pyspark.sql.dataframe.DataFrame

In [10]:
df.head(2)

[Row(Name='Mousumi', Age=50, Experience=20, Salary=30000),
 Row(Name='Prerona', Age=None, Experience=None, Salary=40000)]

In [23]:
df.na.drop(how="all").show()

+---------+----+----------+------+
|     Name| Age|Experience|Salary|
+---------+----+----------+------+
|  Mousumi|  50|        20| 30000|
|    Nahid|  22|         5| 10000|
|Salauddin|  24|        12| 50000|
|   Durjoy|  18|      null|  null|
|     Asha|null|      null|  null|
|   Mafiul|  27|        14| 60000|
+---------+----+----------+------+



# Main Lab

In [37]:
df = spark.read.format('csv').option("header","true").option("inferschema","true").option("mode","failfast").load("lab3.csv")

In [38]:
df.na.drop().show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|  Mousumi| 50|        20| 30000|
|    Nahid| 22|         5| 10000|
|Salauddin| 24|        12| 50000|
|   Mafiul| 27|        14| 60000|
+---------+---+----------+------+



In [39]:
df.na.drop(how="all").show()

+---------+----+----------+------+
|     Name| Age|Experience|Salary|
+---------+----+----------+------+
|  Mousumi|  50|        20| 30000|
|  Prerona|null|      null| 40000|
|    Nahid|  22|         5| 10000|
|Salauddin|  24|        12| 50000|
|   Durjoy|  18|      null|  null|
|     Asha|null|      null|  null|
|   Mafiul|  27|        14| 60000|
+---------+----+----------+------+



In [40]:
df.na.drop(how="any").show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|  Mousumi| 50|        20| 30000|
|    Nahid| 22|         5| 10000|
|Salauddin| 24|        12| 50000|
|   Mafiul| 27|        14| 60000|
+---------+---+----------+------+



In [24]:
df.na.drop(how="any", thresh=2).show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|  Mousumi| 50|        20| 30000|
|    Nahid| 22|         5| 10000|
|Salauddin| 24|        12| 50000|
|   Durjoy| 18|      null|  null|
|   Mafiul| 27|        14| 60000|
+---------+---+----------+------+



In [25]:
df.na.drop(how = "any", subset=["Experience"]).show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|  Mousumi| 50|        20| 30000|
|    Nahid| 22|         5| 10000|
|Salauddin| 24|        12| 50000|
|   Mafiul| 27|        14| 60000|
+---------+---+----------+------+



In [46]:
df = spark.read.format('csv').option("header","true").option("inferschema","true").option("mode","failfast").load("lab3.csv")

In [47]:
df.show()

+---------+----+----------+------+
|     Name| Age|Experience|Salary|
+---------+----+----------+------+
|  Mousumi|  50|        20| 30000|
|  Prerona|null|      null| 40000|
|    Nahid|  22|         5| 10000|
|Salauddin|  24|        12| 50000|
|   Durjoy|  18|      null|  null|
|     Asha|null|      null|  null|
|   Mafiul|  27|        14| 60000|
+---------+----+----------+------+



In [50]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ["Age", "Experience", "Salary"],
    outputCols = ["{}_imputed".format(c) for c in ["Age", "Experience", "Salary"]]

).setStrategy("mean")

In [52]:
# Add imputation cols to df
imputer.fit(df).transform(df).show()

+---------+----+----------+------+-----------+------------------+--------------+
|     Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+---------+----+----------+------+-----------+------------------+--------------+
|  Mousumi|  50|        20| 30000|         50|                20|         30000|
|  Prerona|null|      null| 40000|         28|                12|         40000|
|    Nahid|  22|         5| 10000|         22|                 5|         10000|
|Salauddin|  24|        12| 50000|         24|                12|         50000|
|   Durjoy|  18|      null|  null|         18|                12|         38000|
|     Asha|null|      null|  null|         28|                12|         38000|
|   Mafiul|  27|        14| 60000|         27|                14|         60000|
+---------+----+----------+------+-----------+------------------+--------------+



# Lab evaluation

In [53]:
# Upload the file
from google.colab import files
files.upload()

Saving labtask.csv to labtask.csv


{'labtask.csv': b'Name,Age,Experience,Expected Salary,Salary\nAiyub,22,1,100000,30000\nAli,23,2,50000,\nAman,22,2,,10000\nsourov,24,1,34000,50000\ntuhin,,1,35000,\nAsha,23,,45000,\nAkash,27,2,,60000\nPaul,24,,34000,\nMim,23,2,,34000\nKhadija,,3,60000,25000\nameen,24,2,70000,23000\nJoynal,25,,100000,\nNahid,22,1,120000,23500\nsadi,23,1,340000,12000\nEmon,21,2,,70000\nNur,,2,500000,\nEkbal,24,,500000,40000\nJahid,34,12,450000,\nDulu,23,2,20000,10000\nBabu,27,4,,34000\nRobiul,31,5,600000,\nSoyaeb,34,6,500000,45000\nArju,21,2,500000,3000\nRabeya,22,1,100000,34000\n'}

In [54]:
df = spark.read.format('csv').option("header","true").option("inferschema","true").option("mode","failfast").load("labtask.csv")

In [55]:
df.show()

+-------+----+----------+---------------+------+
|   Name| Age|Experience|Expected Salary|Salary|
+-------+----+----------+---------------+------+
|  Aiyub|  22|         1|         100000| 30000|
|    Ali|  23|         2|          50000|  null|
|   Aman|  22|         2|           null| 10000|
| sourov|  24|         1|          34000| 50000|
|  tuhin|null|         1|          35000|  null|
|   Asha|  23|      null|          45000|  null|
|  Akash|  27|         2|           null| 60000|
|   Paul|  24|      null|          34000|  null|
|    Mim|  23|         2|           null| 34000|
|Khadija|null|         3|          60000| 25000|
|  ameen|  24|         2|          70000| 23000|
| Joynal|  25|      null|         100000|  null|
|  Nahid|  22|         1|         120000| 23500|
|   sadi|  23|         1|         340000| 12000|
|   Emon|  21|         2|           null| 70000|
|    Nur|null|         2|         500000|  null|
|  Ekbal|  24|      null|         500000| 40000|
|  Jahid|  34|      

In [56]:
df.head(5)

[Row(Name='Aiyub', Age=22, Experience=1, Expected Salary=100000, Salary=30000),
 Row(Name='Ali', Age=23, Experience=2, Expected Salary=50000, Salary=None),
 Row(Name='Aman', Age=22, Experience=2, Expected Salary=None, Salary=10000),
 Row(Name='sourov', Age=24, Experience=1, Expected Salary=34000, Salary=50000),
 Row(Name='tuhin', Age=None, Experience=1, Expected Salary=35000, Salary=None)]

In [57]:
df.tail(5)

[Row(Name='Babu', Age=27, Experience=4, Expected Salary=None, Salary=34000),
 Row(Name='Robiul', Age=31, Experience=5, Expected Salary=600000, Salary=None),
 Row(Name='Soyaeb', Age=34, Experience=6, Expected Salary=500000, Salary=45000),
 Row(Name='Arju', Age=21, Experience=2, Expected Salary=500000, Salary=3000),
 Row(Name='Rabeya', Age=22, Experience=1, Expected Salary=100000, Salary=34000)]

In [60]:
df.describe()

DataFrame[summary: string, Name: string, Age: string, Experience: string, Expected Salary: string, Salary: string]

In [76]:
df.columns

['Name', 'Age', 'Experience', 'Expected Salary', 'Salary']

In [65]:
df.na.drop().show()

+------+---+----------+---------------+------+
|  Name|Age|Experience|Expected Salary|Salary|
+------+---+----------+---------------+------+
| Aiyub| 22|         1|         100000| 30000|
|sourov| 24|         1|          34000| 50000|
| ameen| 24|         2|          70000| 23000|
| Nahid| 22|         1|         120000| 23500|
|  sadi| 23|         1|         340000| 12000|
|  Dulu| 23|         2|          20000| 10000|
|Soyaeb| 34|         6|         500000| 45000|
|  Arju| 21|         2|         500000|  3000|
|Rabeya| 22|         1|         100000| 34000|
+------+---+----------+---------------+------+



In [67]:
df.na.drop(how = "any").show()

+------+---+----------+---------------+------+
|  Name|Age|Experience|Expected Salary|Salary|
+------+---+----------+---------------+------+
| Aiyub| 22|         1|         100000| 30000|
|sourov| 24|         1|          34000| 50000|
| ameen| 24|         2|          70000| 23000|
| Nahid| 22|         1|         120000| 23500|
|  sadi| 23|         1|         340000| 12000|
|  Dulu| 23|         2|          20000| 10000|
|Soyaeb| 34|         6|         500000| 45000|
|  Arju| 21|         2|         500000|  3000|
|Rabeya| 22|         1|         100000| 34000|
+------+---+----------+---------------+------+



In [68]:
df.na.drop(how = "all").show()

+-------+----+----------+---------------+------+
|   Name| Age|Experience|Expected Salary|Salary|
+-------+----+----------+---------------+------+
|  Aiyub|  22|         1|         100000| 30000|
|    Ali|  23|         2|          50000|  null|
|   Aman|  22|         2|           null| 10000|
| sourov|  24|         1|          34000| 50000|
|  tuhin|null|         1|          35000|  null|
|   Asha|  23|      null|          45000|  null|
|  Akash|  27|         2|           null| 60000|
|   Paul|  24|      null|          34000|  null|
|    Mim|  23|         2|           null| 34000|
|Khadija|null|         3|          60000| 25000|
|  ameen|  24|         2|          70000| 23000|
| Joynal|  25|      null|         100000|  null|
|  Nahid|  22|         1|         120000| 23500|
|   sadi|  23|         1|         340000| 12000|
|   Emon|  21|         2|           null| 70000|
|    Nur|null|         2|         500000|  null|
|  Ekbal|  24|      null|         500000| 40000|
|  Jahid|  34|      

In [69]:
df.na.drop(how="any", thresh=2).show()

+-------+----+----------+---------------+------+
|   Name| Age|Experience|Expected Salary|Salary|
+-------+----+----------+---------------+------+
|  Aiyub|  22|         1|         100000| 30000|
|    Ali|  23|         2|          50000|  null|
|   Aman|  22|         2|           null| 10000|
| sourov|  24|         1|          34000| 50000|
|  tuhin|null|         1|          35000|  null|
|   Asha|  23|      null|          45000|  null|
|  Akash|  27|         2|           null| 60000|
|   Paul|  24|      null|          34000|  null|
|    Mim|  23|         2|           null| 34000|
|Khadija|null|         3|          60000| 25000|
|  ameen|  24|         2|          70000| 23000|
| Joynal|  25|      null|         100000|  null|
|  Nahid|  22|         1|         120000| 23500|
|   sadi|  23|         1|         340000| 12000|
|   Emon|  21|         2|           null| 70000|
|    Nur|null|         2|         500000|  null|
|  Ekbal|  24|      null|         500000| 40000|
|  Jahid|  34|      

In [71]:
df.na.drop(how = "any", subset=["Salary"]).show()

+-------+----+----------+---------------+------+
|   Name| Age|Experience|Expected Salary|Salary|
+-------+----+----------+---------------+------+
|  Aiyub|  22|         1|         100000| 30000|
|   Aman|  22|         2|           null| 10000|
| sourov|  24|         1|          34000| 50000|
|  Akash|  27|         2|           null| 60000|
|    Mim|  23|         2|           null| 34000|
|Khadija|null|         3|          60000| 25000|
|  ameen|  24|         2|          70000| 23000|
|  Nahid|  22|         1|         120000| 23500|
|   sadi|  23|         1|         340000| 12000|
|   Emon|  21|         2|           null| 70000|
|  Ekbal|  24|      null|         500000| 40000|
|   Dulu|  23|         2|          20000| 10000|
|   Babu|  27|         4|           null| 34000|
| Soyaeb|  34|         6|         500000| 45000|
|   Arju|  21|         2|         500000|  3000|
| Rabeya|  22|         1|         100000| 34000|
+-------+----+----------+---------------+------+



In [91]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ["Age", "Experience", "Expected Salary", "Salary"],
    outputCols = ["{}_imputed".format(c) for c in ["Age", "Experience", "Expected Salary", "Salary"]]

).setStrategy("mean")

In [92]:
imputer.fit(df).transform(df).show()

+-------+----+----------+---------------+------+-----------+------------------+-----------------------+--------------+
|   Name| Age|Experience|Expected Salary|Salary|Age_imputed|Experience_imputed|Expected Salary_imputed|Salary_imputed|
+-------+----+----------+---------------+------+-----------+------------------+-----------------------+--------------+
|  Aiyub|  22|         1|         100000| 30000|         22|                 1|                 100000|         30000|
|    Ali|  23|         2|          50000|  null|         23|                 2|                  50000|         31468|
|   Aman|  22|         2|           null| 10000|         22|                 2|                 218842|         10000|
| sourov|  24|         1|          34000| 50000|         24|                 1|                  34000|         50000|
|  tuhin|null|         1|          35000|  null|         24|                 1|                  35000|         31468|
|   Asha|  23|      null|          45000|  null|

In [93]:
df2 = imputer.fit(df).transform(df)
df2 = df2.drop("Age", "Experience", "Expected Salary", "Salary")
df2.show()

+-------+-----------+------------------+-----------------------+--------------+
|   Name|Age_imputed|Experience_imputed|Expected Salary_imputed|Salary_imputed|
+-------+-----------+------------------+-----------------------+--------------+
|  Aiyub|         22|                 1|                 100000|         30000|
|    Ali|         23|                 2|                  50000|         31468|
|   Aman|         22|                 2|                 218842|         10000|
| sourov|         24|                 1|                  34000|         50000|
|  tuhin|         24|                 1|                  35000|         31468|
|   Asha|         23|                 2|                  45000|         31468|
|  Akash|         27|                 2|                 218842|         60000|
|   Paul|         24|                 2|                  34000|         31468|
|    Mim|         23|                 2|                 218842|         34000|
|Khadija|         24|                 3|

In [94]:
imputer = Imputer(
    inputCols = ["Age", "Experience", "Expected Salary", "Salary"],
    outputCols = ["{}_imputed".format(c) for c in ["Age", "Experience", "Expected Salary", "Salary"]]

).setStrategy("median")
imputer.fit(df).transform(df).show()

+-------+----+----------+---------------+------+-----------+------------------+-----------------------+--------------+
|   Name| Age|Experience|Expected Salary|Salary|Age_imputed|Experience_imputed|Expected Salary_imputed|Salary_imputed|
+-------+----+----------+---------------+------+-----------+------------------+-----------------------+--------------+
|  Aiyub|  22|         1|         100000| 30000|         22|                 1|                 100000|         30000|
|    Ali|  23|         2|          50000|  null|         23|                 2|                  50000|         30000|
|   Aman|  22|         2|           null| 10000|         22|                 2|                 100000|         10000|
| sourov|  24|         1|          34000| 50000|         24|                 1|                  34000|         50000|
|  tuhin|null|         1|          35000|  null|         23|                 1|                  35000|         30000|
|   Asha|  23|      null|          45000|  null|

In [95]:
df3 = imputer.fit(df).transform(df)
df3 = df3.drop("Age", "Experience", "Expected Salary", "Salary")
df3.show()

+-------+-----------+------------------+-----------------------+--------------+
|   Name|Age_imputed|Experience_imputed|Expected Salary_imputed|Salary_imputed|
+-------+-----------+------------------+-----------------------+--------------+
|  Aiyub|         22|                 1|                 100000|         30000|
|    Ali|         23|                 2|                  50000|         30000|
|   Aman|         22|                 2|                 100000|         10000|
| sourov|         24|                 1|                  34000|         50000|
|  tuhin|         23|                 1|                  35000|         30000|
|   Asha|         23|                 2|                  45000|         30000|
|  Akash|         27|                 2|                 100000|         60000|
|   Paul|         24|                 2|                  34000|         30000|
|    Mim|         23|                 2|                 100000|         34000|
|Khadija|         23|                 3|

In [97]:
imputer = Imputer(
    inputCols = ["Age", "Experience", "Expected Salary", "Salary"],
    outputCols = ["{}_imputed".format(c) for c in ["Age", "Experience", "Expected Salary", "Salary"]]

).setStrategy("mode")
imputer.fit(df).transform(df).show()

+-------+----+----------+---------------+------+-----------+------------------+-----------------------+--------------+
|   Name| Age|Experience|Expected Salary|Salary|Age_imputed|Experience_imputed|Expected Salary_imputed|Salary_imputed|
+-------+----+----------+---------------+------+-----------+------------------+-----------------------+--------------+
|  Aiyub|  22|         1|         100000| 30000|         22|                 1|                 100000|         30000|
|    Ali|  23|         2|          50000|  null|         23|                 2|                  50000|         34000|
|   Aman|  22|         2|           null| 10000|         22|                 2|                 500000|         10000|
| sourov|  24|         1|          34000| 50000|         24|                 1|                  34000|         50000|
|  tuhin|null|         1|          35000|  null|         23|                 1|                  35000|         34000|
|   Asha|  23|      null|          45000|  null|

In [98]:
df4 = imputer.fit(df).transform(df)
df4 = df4.drop("Age", "Experience", "Expected Salary", "Salary")
df4.show()

+-------+-----------+------------------+-----------------------+--------------+
|   Name|Age_imputed|Experience_imputed|Expected Salary_imputed|Salary_imputed|
+-------+-----------+------------------+-----------------------+--------------+
|  Aiyub|         22|                 1|                 100000|         30000|
|    Ali|         23|                 2|                  50000|         34000|
|   Aman|         22|                 2|                 500000|         10000|
| sourov|         24|                 1|                  34000|         50000|
|  tuhin|         23|                 1|                  35000|         34000|
|   Asha|         23|                 2|                  45000|         34000|
|  Akash|         27|                 2|                 500000|         60000|
|   Paul|         24|                 2|                  34000|         34000|
|    Mim|         23|                 2|                 500000|         34000|
|Khadija|         23|                 3|